|<h2>Book:</h2>|<h1><a href="https://open.substack.com/pub/mikexcohen/p/llm-breakdown-16-tokenization-words" target="_blank">50 ML projects to understand LLMs</a></h1>|
|-|:-:|
|<h2>Project:</h2>|<h1><b>[45] Minkowski distance, mutual information, and token positions</b></h1>|
|<h2>Author:<h2>|<h1>Mike X Cohen, <a href="https://sincxpress.com" target="_blank">sincxpress.com</a></h1>|

<br>

<i>Using the code without reading the book may lead to confusion or errors.</i>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

!pip install wikipedia
import wikipedia

# stats library for kendall correlation (when one variable is ordinal [sorted categorical])
import scipy.stats as stats

from sklearn.feature_selection import mutual_info_regression
from scipy.spatial.distance import minkowski

import torch
from transformers import AutoModelForCausalLM,GPT2Tokenizer

In [ ]:
### matplotlib adjustments (commented lines are for dark mode)

# svg plots (higher-res)
import matplotlib_inline.backend_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('svg')

plt.rcParams.update({
    # 'figure.facecolor': '#282a2c',
    # 'figure.edgecolor': '#282a2c',
    # 'axes.facecolor':   '#282a2c',
    # 'axes.edgecolor':   '#DDE2F4',
    # 'axes.labelcolor':  '#DDE2F4',
    # 'xtick.color':      '#DDE2F4',
    # 'ytick.color':      '#DDE2F4',
    # 'text.color':       '#DDE2F4',
    'axes.spines.right': False,
    'axes.spines.top':   False,
    'axes.titleweight': 'bold',
    'axes.labelweight': 'bold',
    'savefig.dpi':300
})

# **Part 1: Minkowski distances**

In [ ]:
# generate some data
x = np.random.randn(100)
y = np.random.randn(100)

# implement the distance measure
for p in range(1,3):

  # manual calculation
  mink_man = np.sum( abs(x-y)**p )**(1/p)

  # via scipy
  mink_sp = minkowski(x,y,p)

  # show their equivalence
  print(f'p = {p}:\n manual: {mink_man:.3f}\n  scipy: {mink_sp:.3f}\n')

# **Part 2: Mutual information: manual and scikit-learn**

In [ ]:
# the data
N = 347
x = np.random.uniform(low=.001,high=2.7,size=N)
y = np.cos(2*x)*3 + np.log(x) + np.random.normal(0,.5,N)


# 2D histogram
Z,xx,yy = np.histogram2d(x,y,bins=8)

_,axs = plt.subplots(1,2,figsize=(12,4))
axs[0].plot(x,y,'ko',markersize=8,markerfacecolor=[.7,.7,.9,.3])
axs[0].set(xlabel='x',ylabel='y',title='A) Full resolution data')

h = axs[1].imshow(Z.T,extent=[xx[0],xx[-1],yy[0],yy[-1]],vmin=0,vmax=Z.max()*.7,origin='lower',aspect='auto',cmap='magma')
axs[1].set(xlabel='x',ylabel='y',title='B) Discretized (binned) data')
axs[1].plot(x,y,'ko',markerfacecolor=[.7,.7,.9,.5],markersize=8)
plt.colorbar(h,ax=axs[1],pad=.01,label='Count')
plt.suptitle('Z,xx,yy = np.histogram2d(x,y,bins=8)', fontfamily='monospace')

plt.tight_layout()
plt.savefig('ch7_proj45_part2a.png')
plt.show()

In [ ]:
eps = 1e-13

# joint entropy from proportion
p_Z = Z / Z.sum()
entropy_Z = -np.sum( p_Z * np.log2(p_Z+eps) )

# single-variable entropies
p_x = np.sum(p_Z, axis=1)
entropy_x = -np.sum( p_x * np.log2(p_x+eps) )
p_y = np.sum(p_Z, axis=0)
entropy_y = -np.sum( p_y * np.log2(p_y+eps) )

# mutual information via direct translation of the formula
miMan = (entropy_x+entropy_y) - entropy_Z

print(f'Mutual information (manual) : {miMan:.2f}')

In [ ]:
# via scikit-learn's MI function optimized for continuous variables
miSk = mutual_info_regression(x.reshape(-1,1),y)[0]
print(f'Mutual information (scikit-learn): {miSk:.2f}')

In [ ]:
# impact of discretization

bincounts = np.arange(4,25)
mi_by_bincount = np.zeros(len(bincounts))

for i in range(len(mi_by_bincount)):

  Z,xx,yy = np.histogram2d(x,y,bins=bincounts[i])

  # proportion via sum-scaling
  p_Z = Z / Z.sum()
  p_x = np.sum(p_Z, axis=1)
  p_y = np.sum(p_Z, axis=0)

  # calculate entropy
  entropy_x = -np.sum( p_x * np.log2(p_x+eps) )
  entropy_y = -np.sum( p_y * np.log2(p_y+eps) )

  # MI as difference of entropies
  entropy_Z = -np.sum( p_Z * np.log2(p_Z+eps) )
  mi_by_bincount[i] = (entropy_x+entropy_y) - entropy_Z


plt.figure(figsize=(8,4))
plt.axhline(miSk,color='r',linestyle='--',linewidth=2,label='scikit-learn')
plt.plot(bincounts,mi_by_bincount,'kh',markersize=12,markerfacecolor=[.7,.7,.9],label='Manual')
plt.gca().set(xticks=bincounts[::2],xlabel='Number of bins',ylabel='Mutual information')
plt.legend()

plt.tight_layout()
plt.savefig('ch7_proj45_part2b.png')
plt.show()

In [ ]:
# correlation coefficient
np.corrcoef(x,y)

# **Part 3: Model, hooks, tokens, and activations**

In [ ]:
# load pretrained GPT-2 model and tokenizer
gpt2 = AutoModelForCausalLM.from_pretrained('gpt2-large')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-large')

# variable for the number of transformer layers
nLayers = len(gpt2.transformer.h)

gpt2.eval()

In [ ]:
# hook function to store attention and MLP projections (see also part5_neuron_hookVsHiddenStates.ipynb)
activations = {}

def implant_hook_attn(layer_number):
  def hook(module, input, output):
    activations[f'att_{layer_number}'] = output.detach().numpy()
  return hook

# and mlp layers
def implant_hook_mlp(layer_number):
  def hook(module, input, output):
    activations[f'mlp_{layer_number}'] = output.detach().numpy()
  return hook

# implant hooks
handles = []
for layeri in range(nLayers):
  h1 = gpt2.transformer.h[layeri].attn.c_proj.register_forward_hook(implant_hook_attn(layeri))
  h2 = gpt2.transformer.h[layeri].mlp.c_proj.register_forward_hook(implant_hook_mlp(layeri))

  handles.append(h1)
  handles.append(h2)

In [ ]:
# from https://en.wikipedia.org/wiki/Turkish_coffee
text = wikipedia.page('Turkish_coffee').content
tokens = tokenizer.encode(text,return_tensors='pt')
print(f'There are {len(tokens[0])} tokens, {len(set(tokens[0].tolist()))} of which are unique.')

In [ ]:
# find all the "coffee" target indices
target = ' coffee'
target_idxs = torch.where(tokens[0,:]==tokenizer.encode(target)[0])[0]

# just the first 10
n_targets = 10
target_idxs = target_idxs[:n_targets]
target_idxs

In [ ]:
# forward pass to get activations
with torch.no_grad():
  gpt2(tokens[:,:target_idxs[-1]+1])

In [ ]:
for k,v in activations.items():
  print(f'{k} has shape {v.shape}')

# **Part 4: Relationships between distance measures**

In [ ]:
distances = np.zeros((n_targets,n_targets,3))
tokdists = np.zeros((n_targets,n_targets))

# double-loop over the word pairs
for toki in range(n_targets):
  for tokj in range(toki+1,n_targets):

    # extract the data
    x = activations['mlp_3'][0,target_idxs[toki],:]
    y = activations['mlp_3'][0,target_idxs[tokj],:]

    # pairwise distance measures
    distances[toki,tokj,0] = mutual_info_regression(x[:,None],y)[0]
    distances[toki,tokj,1] = minkowski(x,y,1)
    distances[toki,tokj,2] = minkowski(x,y,2)

    tokdists[toki,tokj] = target_idxs[tokj]-target_idxs[toki]

In [ ]:
# indices of nonzero values
triu_idx = np.triu_indices(n_targets,k=1)

_,axs = plt.subplots(1,2,figsize=(10,3.5))

# without normalization
axs[0].plot(distances[triu_idx[0],triu_idx[1],0],
            distances[triu_idx[0],triu_idx[1],1],'ro',
            markerfacecolor=[.9,.7,.7,.7],markersize=8,label='MI vs. $L_1$')
axs[0].plot(distances[triu_idx[0],triu_idx[1],0],
            distances[triu_idx[0],triu_idx[1],2],'gs',
            markerfacecolor=[.7,.9,.7,.7],markersize=8,label='MI vs. $L_2$')
axs[0].plot(distances[triu_idx[0],triu_idx[1],1],
            distances[triu_idx[0],triu_idx[1],2],'bh',
            markerfacecolor=[.7,.7,.9,.7],markersize=8,label='$L_1$ vs. $L_2$')


# with normalization
mi = distances[triu_idx[0],triu_idx[1],0]
mi /= max(mi)
L1 = distances[triu_idx[0],triu_idx[1],1]
L1 /= max(L1)
L2 = distances[triu_idx[0],triu_idx[1],2]
L2 /= max(L2)

axs[1].plot(mi,L1,'ro',markerfacecolor=[.9,.7,.7,.7],markersize=8,label='MI vs. $L_1$')
axs[1].plot(mi,L2,'gs',markerfacecolor=[.7,.9,.7,.7],markersize=8,label='MI vs. $L_2$')
axs[1].plot(L1,L2,'bh',markerfacecolor=[.7,.7,.9,.7],markersize=8,label='$L_1$ vs. $L_2$')

axs[0].set(xlabel='Measure',ylabel='Measure',title='A) Non-normalized')
axs[1].set(xlabel='Measure (norm.)',ylabel='Measure (norm.)',title='B) Max-value normalized')
axs[0].legend()
axs[1].legend()

plt.tight_layout()
plt.savefig('ch7_proj45_part4.png')
plt.show()

# **Part 5: Projection distances by position distances**

In [ ]:
fig,axs = plt.subplots(2,4,figsize=(11,5))

metric_labels = [ 'Mut. Info','Mink. $L_1$','Mink. $L_2$' ]

for disti in range(3):

  # this distance measure
  D = distances[:,:,disti]

  # show the distance heatmap
  h = axs[0,disti].imshow(D,origin='lower',aspect='auto',vmin=0,vmax=D.max()*.8,cmap='magma')
  axs[0,disti].set(xlabel='Target token index',ylabel='Target token index',title=f'{"ABC"[disti]}) {metric_labels[disti]}')
  fig.colorbar(h,ax=axs[0,disti],pad=.02,fraction=.047)

  # correlate vector distance with token distance
  r = stats.kendalltau(tokdists[triu_idx],D[triu_idx])

  # scatter plot
  axs[1,disti].plot(tokdists[triu_idx],D[triu_idx],'ks',markersize=6,markerfacecolor=[.7,.7,.9,.7])
  axs[1,disti].set(xlabel='Position distance',ylabel=f'{metric_labels[disti]}',
            title=f"{'EFG'[disti]}) $\\mathbf{{\\tau}}={r.statistic:.2f}\\; (p={r.pvalue:.3f})$")


# inter-token position differences
h = axs[0,3].imshow(tokdists,origin='lower',aspect='auto',vmin=0,vmax=tokdists.max()*.8,cmap='magma')
axs[0,3].set(xlabel='Target token index',ylabel='Target token index',title=f'D) Position dists.')
fig.colorbar(h,ax=axs[0,3],pad=.02,fraction=.047)
axs[1,3].axis('off')

plt.tight_layout()
plt.savefig('ch7_proj45_part5.png')
plt.show()

# **Part 6: Laminar sweep of correlations**

In [ ]:
summary_results = np.zeros((2,nLayers,2,3))

# initialize temp matrices (overwritten in each layer)
dists_A = np.zeros((n_targets,n_targets,3)) # attention
dists_M = np.zeros((n_targets,n_targets,3)) # MLP


# loop over layers
for layeri in range(nLayers):

  # double-loop over the word pairs
  for toki in range(n_targets):
    for tokj in range(toki+1,n_targets):

      ### ATTENTION block
      # extract the data
      x = activations[f'att_{layeri}'][0,target_idxs[toki],:]
      y = activations[f'att_{layeri}'][0,target_idxs[tokj],:]

      # pairwise distance measures
      dists_A[toki,tokj,0] = mutual_info_regression(x[:,None],y)[0]
      dists_A[toki,tokj,1] = minkowski(x,y,1)
      dists_A[toki,tokj,2] = minkowski(x,y,2)


      ### MLP block
      # extract the data
      x = activations[f'mlp_{layeri}'][0,target_idxs[toki],:]
      y = activations[f'mlp_{layeri}'][0,target_idxs[tokj],:]

      # pairwise distance measures
      dists_M[toki,tokj,0] = mutual_info_regression(x[:,None],y)[0]
      dists_M[toki,tokj,1] = minkowski(x,y,1)
      dists_M[toki,tokj,2] = minkowski(x,y,2)


  # inter-token distances (doesn't change for attn-vs-mlp)
  uDi = tokdists[triu_idx]

  # loop over distance measures
  for i in range(3):

    # ATTENTION summary statistics
    vals = dists_A[triu_idx[0],triu_idx[1],i]
    summary_results[0,layeri,0,i] = np.mean(vals)
    summary_results[0,layeri,1,i] = stats.kendalltau(vals,uDi).statistic

    # MLP summary statistics
    vals = dists_M[triu_idx[0],triu_idx[1],i]
    summary_results[1,layeri,0,i] = np.mean(vals)
    summary_results[1,layeri,1,i] = stats.kendalltau(vals,uDi).statistic

In [ ]:
_,axs = plt.subplots(2,2,figsize=(12,6))

c = 'rgb'
s = 'osh'

for i in range(3):

  # Attention
  d = summary_results[0,:,0,i]
  axs[0,0].plot(d/d.max(),f'{c[i]}{s[i]}-',linewidth=.3,alpha=.7,label=metric_labels[i])
  axs[0,1].plot(summary_results[0,:,1,i],f'{c[i]}{s[i]}-',linewidth=.3,alpha=.7,label=metric_labels[i])

  # MLP
  d = summary_results[1,:,0,i]
  axs[1,0].plot(d/d.max(),f'{c[i]}{s[i]}-',linewidth=.3,alpha=.7,label=metric_labels[i])
  axs[1,1].plot(summary_results[1,:,1,i],f'{c[i]}{s[i]}-',linewidth=.3,alpha=.7,label=metric_labels[i])
  # note: flip the sign of MI by scaling summary_results by [-1,1,1][i]
  #       or just plot abs(summary_results)

# horizontal line at r=0
axs[0,1].axhline(0,color='k',linestyle='--',linewidth=.3)
axs[1,1].axhline(0,color='k',linestyle='--',linewidth=.3)


# adjustments
for a in axs.flatten(): a.legend()
axs[0,0].set(xlabel='Transformer layer',ylabel='Mean distance (max-norm)',title='A) ATTENTION: Average distances')
axs[0,1].set(xlabel='Transformer layer',ylabel='Correlation coefficient',title='B) ATTENTION: Correlations with position')
axs[1,0].set(xlabel='Transformer layer',ylabel='Mean distance (max-norm)',title='C) MLP: Average distances')
axs[1,1].set(xlabel='Transformer layer',ylabel='Correlation coefficient',title='D) MLP: Correlations with position')


plt.tight_layout()
plt.savefig('ch7_proj45_part6.png')
plt.show()

# **Part 7: A standard application of mutual information**

In [ ]:
# batch size parameters
n_sequences = 100
n_context = 9 # actually this + 1 b/c context is before target

# get the target indices again
target_idxs = torch.where(tokens==tokenizer.encode(target)[0])[1]

# remove any that are too early to have a full context
target_idxs = target_idxs[target_idxs>n_context+1]

n_targets = len(target_idxs)
n_targets

In [ ]:
batch = torch.zeros((n_sequences,n_context+1),dtype=torch.long)

for i in range(n_sequences):
  start = target_idxs[i]-n_context
  stop  = target_idxs[i]+1
  batch[i,:] = tokens[0,start:stop]

batch

In [ ]:
with torch.no_grad(): gpt2(batch)

In [ ]:
for k,v in activations.items():
  print(f'key "{k}" has shape {v.shape}')

In [ ]:
mi = np.zeros((nLayers,nLayers,3))

for i in range(nLayers):
  for j in range(i+1,nLayers):

    # mutual information across attention projection vector norms
    normi = np.linalg.norm(activations[f'att_{i}'][:,-1,:],axis=-1)
    normj = np.linalg.norm(activations[f'att_{j}'][:,-1,:],axis=-1)
    mi[i,j,0] = mutual_info_regression(normi[:,None],normj).item()

    # mutual information across MLP projection vector norms
    normi = np.linalg.norm(activations[f'mlp_{i}'][:,-1,:],axis=-1)
    normj = np.linalg.norm(activations[f'mlp_{j}'][:,-1,:],axis=-1)
    mi[i,j,1] = mutual_info_regression(normi[:,None],normj).item()

    # inter-layer distances
    mi[i,j,2] = j-i

In [ ]:
_,axs = plt.subplots(2,3,figsize=(12,7))

# mutual information matrices
axs[0,0].imshow(mi[:,:,0],aspect='auto',vmin=0,vmax=.3,cmap='magma')
axs[0,1].imshow(mi[:,:,1],aspect='auto',vmin=0,vmax=.3,cmap='magma')
axs[0,2].imshow(mi[:,:,2],aspect='auto',cmap='magma')

# scatter plots
axs[1,0].plot(mi[:,:,2][np.triu_indices_from(mi[:,:,2],k=1)],
              mi[:,:,0][np.triu_indices_from(mi[:,:,2],k=1)],'ro',markerfacecolor=[.9,.7,.7,.5])
axs[1,1].plot(mi[:,:,2][np.triu_indices_from(mi[:,:,2],k=1)],
              mi[:,:,1][np.triu_indices_from(mi[:,:,2],k=1)],'bs',markerfacecolor=[.7,.7,.9,.5])
axs[1,2].plot(mi[:,:,0][np.triu_indices_from(mi[:,:,2],k=1)],
              mi[:,:,1][np.triu_indices_from(mi[:,:,2],k=1)],'gh',markerfacecolor=[.7,.9,.7,.5])

# plot adjustments
axs[0,0].set(xlabel='Layer index',ylabel='Layer index',title='A) Attention: Pairwise mutual info')
axs[0,1].set(xlabel='Layer index',ylabel='Layer index',title='B) MLP: Pairwise mutual info')
axs[0,2].set(xlabel='Layer index',ylabel='Layer index',title='C) Inter-layer distances')

axs[1,0].set(xlabel='Inter-layer distance',ylabel='Mutual information',title='D) Attention: MI with distance')
axs[1,1].set(xlabel='Inter-layer distance',ylabel='Mutual information',title='E) MLP: MI with distance')
axs[1,2].set(xlabel='Attention mutual information',ylabel='MLP mutual information',title='F) MI: Attention vs. MLP')

plt.tight_layout()
plt.savefig('ch7_proj45_part7.png')
plt.show()